In [ ]:
import os

os.environ["HF_HOME"] = "/root/autodl-tmp/HF_download"
os.environ["MODELSCOPE_CACHE"] = "/root/autodl-tmp/MODELSCOPE_download"
os.environ["HF_TOKEN"] = "xxx"
# os.environ["HF_ENDPOINT"] = "https://hf-mirror.com"

In [ ]:
os.environ["http_proxy"] = "http://127.0.0.1:7890"
os.environ["https_proxy"] = "http://127.0.0.1:7890"

In [ ]:
import transformers
print("transformers:", transformers.__version__)

In [ ]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/MODELSCOPE_download/models/ZhipuAI/chatglm3-6b-base", trust_remote_code=True)
tokenizer

In [ ]:
from transformers import AutoModelForCausalLM
import torch

model = AutoModelForCausalLM.from_pretrained(
  "/root/autodl-tmp/MODELSCOPE_download/models/ZhipuAI/chatglm3-6b-base",
  torch_dtype=torch.bfloat16,
  trust_remote_code=True,
  low_cpu_mem_usage=True,
  device_map="auto"
)

In [ ]:
from datasets import Dataset

datasets = Dataset.load_from_disk("/root/autodl-tmp/code/test-transformers/data/alpaca_data_zh")

In [ ]:
def process_function(example):
  MAX_LENGTH = 256
  instruction = "\n".join([example["instruction"].strip(), example["input"].strip()]).strip()
  instruction = tokenizer.build_chat_input(instruction, history=[], role="user")
  response = tokenizer("\n" + example["output"], add_special_tokens=False)
  input_ids = instruction["input_ids"][0].numpy().tolist() + response["input_ids"] + [tokenizer.eos_token_id]
  attention_mask =instruction["attention_mask"][0].numpy().tolist() + response["attention_mask"] + [1]
  labels = [-100] * len(instruction["input_ids"][0]) + response["input_ids"] + [tokenizer.eos_token_id]
  if len(input_ids) > MAX_LENGTH:
    return {}
  return {
    "input_ids": input_ids,
    "attention_mask": attention_mask,
    "labels": labels
  }

In [ ]:
tokenized_datasets = datasets.map(process_function, remove_columns=datasets.column_names)

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType
config = LoraConfig(task_type=TaskType.CAUSAL_LM)

In [ ]:
model = get_peft_model(model, config)

In [ ]:
model.print_trainable_parameters()

In [ ]:
model

In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="/root/autodl-tmp/code/test-transformers/test-kbit/16bits-training/glm3-chatbot2",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    logging_steps=10,
    num_train_epochs=1,
    learning_rate=1e-4,
    # remove_unused_columns=False,
    save_strategy="epoch",
    report_to=["tensorboard"],
)

In [ ]:
from transformers import Trainer, DataCollatorForSeq2Seq

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=tokenized_datasets.select(range(6000)),
    data_collator=DataCollatorForSeq2Seq(tokenizer=tokenizer, padding=True),
)

In [ ]:
trainer.train()

In [ ]:
model.eval()
with torch.inference_mode():
  print(model.chat(tokenizer, "考试的技巧有哪些？", history=[]))